In [ ]:
from funasr import AutoModel
import re
import sys

### 语音转写+端点检测

In [ ]:
path_asr = r"E:\LargeModel\Speech_Synthesis\GPT_Sovits\GPT-SoVITS-beta\GPT-SoVITS-beta_fast_inference_0316\tools\asr\models\speech_paraformer-large_asr_nat-zh-cn-16k-common-vocab8404-pytorch"
path_vad = r"E:\LargeModel\Speech_to_Text\Funasr\iic\speech_fsmn_vad_zh-cn-16k-common-pytorch"
path_punc=r"E:\LargeModel\Speech_to_Text\Funasr\iic\punc_ct-transformer_zh-cn-common-vocab272727-pytorch"
model = AutoModel(
    model               = path_asr,
    model_revision      = "v2.0.4",
    vad_model           = path_vad,
    vad_model_revision  = "v2.0.4",
    # punc_model          = path_punc,
    # punc_model_revision = "v2.0.4",
)

input_file_name = r'D:\Desktop\人声.wav'
text0 = model.generate(input=input_file_name)[0]["text"]
help(AutoModel)

### 标点恢复

In [ ]:
punc_model = AutoModel(model=path_punc, model_revision="v2.0.4")
punc_res = punc_model.generate(input=text0)[0]['text']
print(punc_res)

#每当有端点时在text0的基础上添加一个空格
split = '[,.?!;:。；：？！，\n]'
punc_res_copy =punc_res
punc_text=''
for subtext in text0.split(' '):
    while(len(subtext)>0):
        temp = re.search(split,  punc_res_copy)
        if not temp:
            punc_text += subtext
            punc_text += ' '
            break
        left, right = temp.span()
        if  left > len(subtext):
            right = len(subtext)
            subtext=''
        else:
            subtext = subtext[left:]
        punc_text += punc_res_copy[:right]
        punc_text += ' '
        punc_res_copy = punc_res_copy[right:]
text0 = punc_text
del punc_text, punc_res_copy

### 时间戳检测

In [ ]:
#时间戳后面的文字是否带标点
include_punc=True
no_endpoint = re.sub(split, '', text0).strip()

In [ ]:
model = AutoModel(model="E:\LargeModel\Speech_to_Text\Funasr\iic\speech_timestamp_prediction-v1-16k-offline", model_revision="v2.0.4")
wav_file = input_file_name
# text_file = r"E:\LargeModel\Speech_to_Text\Funasr\yueer.txt"
res = model.generate(input=(wav_file, no_endpoint), data_type=("sound", "text"))

In [ ]:
def time_convert(tsecond):
    '''
    将秒转为时:分:秒,毫秒的形式
    '''
    ms = tsecond-int(tsecond)
    tsecond = int(tsecond)
    hour = tsecond//3600
    minute = tsecond%3600//60
    second = tsecond%60
    return(str(hour).rjust(2,'0')+':'+str(minute).rjust(2,'0')+':'+str(second).rjust(2,'0')+','+str(round(ms*1000)))

#### 字幕和时间轴合并

In [ ]:
# res:list
# res[0]:dict:{key, text, timestamp}
tstamp = res[0]['timestamp']
no_endpoint = no_endpoint.split(' ')
if include_punc:
    stamptext = text0
else:
    stamptext = no_endpoint
stamptext = stamptext.split(' ')
now_stamp=0
num=1
# 在输入音频的文件位置生成一个同名的字幕文件
srt_file = '.'.join(input_file_name.split('.')[:-1])+'.srt'
orign = sys.stdout 
sys.stdout = open(srt_file, 'w')
for index in range(len(no_endpoint)):
    start = tstamp[now_stamp][0]/1000#ms->s
    now_stamp += len(no_endpoint[index])
    end= tstamp[now_stamp-1][1]/1000#ms->s
    print("%d\n%s --> %s\n%s" % (num, time_convert(start), time_convert(end), stamptext[index]))
    num+=1
sys.stdout = orign 

### 说话人检测(检测两个音频是否是同一人说话)

In [ ]:
# TorchAudio is not built with sox extension
from modelscope.pipelines import pipeline
input_file_name = r"E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\自己的模型\姬如千泷\素材\人声.wav"
sv_pipeline = pipeline(
    task='speaker-verification',
    model="E:\LargeModel\Speech_to_Text\Funasr\damo\speech_campplus_sv_zh-cn_16k-common",
    model_revision='v1.0.0'
)
speaker1_a_wav = "E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\派蒙-效果并不好\参考音频\生气—呜哇好生气啊！不要把我跟一斗相提并论！.wav"
speaker1_b_wav = "E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\派蒙-效果并不好\参考音频\说话—既然罗莎莉亚说足迹上有元素力，用元素视野应该能很清楚地看到吧。.wav"
speaker2_a_wav = "E:\LargeModel\Speech_Synthesis\GPT_SOVITS_0训练集\派蒙-效果并不好\参考音频\疑问—哇，这个，还有这个…只是和史莱姆打了一场，就有这么多结论吗？.wav"

result = sv_pipeline([speaker1_a_wav, speaker1_b_wav], thr=0.1)
print(result['text'])

### 情感表征

In [ ]:
input_file_name = r'D:\Desktop\人声.wav'
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

'''
Using the finetuned emotion recognization model
rec_result contains {'feats', 'labels', 'scores'}
	extract_embedding=False: 9-class emotions with scores
	extract_embedding=True: 9-class emotions with scores, along with features

9-class emotions:
    0: angry
    1: disgusted
    2: fearful
    3: happy
    4: neutral
    5: other
    6: sad
    7: surprised
    8: unknown
'''
inference_pipeline = pipeline(
    task=Tasks.emotion_recognition,
    model=r"E:\LargeModel\Speech_to_Text\Funasr\iic\emotion2vec_base_finetuned", model_revision="v2.0.4")

In [ ]:
rec_result = inference_pipeline(input_file_name, output_dir="./outputs", granularity="utterance", extract_embedding=False)
result = list(map(lambda x:(rec_result[0]['labels'][x[0]],x[1]), sorted(enumerate(rec_result[0]['scores']),key=lambda x:x[1], reverse=True)))
print(result[0][0])
for content in result:
    print(*content)